# 1.boosting

boosting is another approach of ensemble learning.

boosting first train a base learner which is rather weak, then adjust that leaner with the knowledge of current prediction to make it stronger.

for example, adjust the traing set distribution, focus on samples that previous predict wrong.

# 2.adaboost

suppose a binary classification problem, dataset is $\left\{(x_{1}, y_{1}),...,({x_{N},y_{N}})\right\}$, $y_{i} \in \left\{-1, 1\right\}$, total $N$ samples.

initial sample distribution is $D_{1} = (\frac{1}{N},...,\frac{1}{N}) = (w_{1,1},...,w_{1,N})$

for $m=1,...,M$:

1. train $G_{m}: \mathcal{X} \rightarrow \left\{-1, 1\right\}$ based on $D_{m} = (w_{m,1},...,w_{m,N})$

2. compute misclassify error:
$$e_{m} = \sum_{i=1}^{N}P(G_{m}(x_{i}) \ne y_{i}) = \sum_{i=1}^{N}w_{m,i}I(G_{m}(x_{i} \ne y_{i}))$$
3. update distribution $D_{m}$ to $D_{m+1}$:
$$
w_{m+1, i} =
\begin{cases}
\frac{w_{m,i}}{Z_{m}} \\
{\frac{1 - e_{m}}{e_{m}}}\frac{w_{m,i}}{Z_{m}}
\end{cases}
$$
where $Z_{m}$ is the normalization factor that makes $D_{m+1}$ a proper distribution.<br>
this step simply enlarge the previous wrong predicted sample's distribution by a factor $\frac{1 - e_{m}}{e_{m}}$.

then construct a linear combination of $G_{m}$:

$$f(x) = \sum_{m=1}^{M}log\ \frac{1 - e_{m}}{e_{m}}G_{m}(x)$$

finally the resulting classifier:

$$G(x) = sign(f(x)) = sign\left(\sum_{i=1}^{N}log\ \frac{1 - e_{m}}{e_{m}}G_{m}(x)\right)$$

# 3.forward stage wise algorithm

consider addictive model:

$$f(x) = \sum_{m=1}^{M}\beta_{m}b(x; \gamma_{m})$$

here $b$ is the base model, $\gamma_{m}$ is it's paramter.

given loss function $L$, we try to solve the following optimization problem:

$$\underset{\beta, \gamma}{min}\sum_{i=1}^{N}L\left(y_{i}, \sum_{m=1}^{M}\beta_{m}b(x; \gamma_{m})\right)$$

this is usually very complex.

foward stage wise algorithm simplify it by going only one stage at a time:

init $f_{0}(x) = 0$.

for $m=1,...,M$:

1. do the following one step optimization:
$$(\beta_{m}, \gamma_{m}) = \underset{\beta,\gamma}{argmin}\sum_{i=1}^{N}L(y_{i}, f_{m-1}(x_{i}) + \beta{b(x_{i},\gamma)})$$
2. update:
$$f_{m}(x) = f_{m-1}(x) + \beta_{m}b(x;\gamma_{m})$$

finally the resulting addictive model:

$$f(x) = f_{M}(x) = \sum_{m=1}^{M}\beta_{m}b(x; \gamma_{m})$$

adaboost $\Leftrightarrow$ foward stage wise algorithm when $L(y, f(x)) = exp(-yf(x))$, t.b.c.

# 4.boosting tree

boosting tree model:

$$f(x) = \sum_{m=1}^{M}T(x; \theta_{m})$$

where $T(x; \theta_{m})$ is a decision tree.

boosting tree use the forward stage wise algorithm.

for binary classification boosting tree, we use $L(y, f(x)) = exp(-yf(x))$, so equivalent to adaboost where base model is decision tree.

for regression problem, we use $L(y, f(x)) = (y - f(x))^2$, i.e square error.

then the forward stage wise algorithm:

$$\hat\theta_{m} = \underset{\theta_{m}}{argmin}\sum_{i=1}^{N}L(y_{i}; f_{m-1}(x) + T(x_{i};\theta_{m}))$$

$$L(y_{i}; f_{m-1}(x) + T(x_{i};\theta_{m})) = [y - f_{m-1}(x) - T(x;\theta_{m})]^{2} = [r - T(x;\theta_{m})]^{2}$$

here $r = y - f_{m-1}(x)$ is the residual.

so for regression tree, we just need to fit the residual!

# 5.GBDT-gradient boosting decision tree

while we can easily proceed in forward stage wise algorithm if loss function is square or exponential, for generic loss, this is not easy.

we use the gradient of the loss function to proceed  in generic case, that is GBDT.

GBDT for regression:

input: training set $T = \left\{(x_{1},y_{1}),...,(x_{N},y_{N})\right\}$, $x_{i} \in \mathcal{X} \in \mathbb{R}^{n}$, $y_{i} \in \mathcal{Y} \in \mathbb{R}$, loss function $L(y, f(x))$.

output: regression tree $\hat{f}(x)$.

init 
$$f_{0}(x) = \underset{c}{argmin}\sum_{i=1}^{N}L(y_{i}, c)$$

for $m=1,...,M$.

1. for $i=1,...,N$, compute
$$r_{mi} = -\left[\frac{\partial{L(y_{i}, f(x_{i}))}}{\partial f(x_{i})}\right]_{f = f_{m-1}}$$

2. generate a tree that fits $r_{mi}$, denote it's leaf nodes areas as $R_{mi},j=1,...,J$.

3. for $c=1,...,J$, compute
$$c_{mj} = \underset{c}{argmin}\sum_{x_{i} \in R_{mj}}L(y_{i}, f_{m-1}(x_{i}) + c)$$

4. update $f_{m}(x) = f_{m-1}(x) + \sum_{j=1}^{J}c_{mj}I(x \in R_{mj})$.

finally the resulting regression tree:

$$\hat{f}(x) = f_{M}(x) = \sum_{m=1}^{M}\sum_{j=1}^{J}c_{mj}I(x \in R_{mj})$$

in one word, while we want to minimize $L(y_{i}, f_{m-1}(x_{i}) + c)$, we just pick it's negative gradient like gradient descent:

$$c = -\left[\frac{\partial{L(y_{i}, f(x_{i}))}}{\partial f(x_{i})}\right]_{f = f_{m-1}}$$